# Stellar Mass by Region

In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

/u/fiza/conda-envs/auriga/lib/python3.7/site-packages/setuptools/_importlib.py:23: UserWarning: `importlib-metadata` version is incompatible with `setuptools`.
This problem is likely to be solved by installing an updated version of `importlib-metadata`.
  warnings.warn(msg)  # Ensure a descriptive message is shown.


In [9]:
from auriga.snapshot import Snapshot
from auriga.images import figure_setup, set_axs_configuration
from auriga.settings import Settings
from auriga.parser import parse

In [3]:
figure_setup()
settings = Settings()

In [4]:
DISC_STD_CIRC = settings.disc_std_circ
DISC_MIN_CIRC = settings.disc_min_circ
COLD_DISC_DELTA_CIRC = settings.cold_disc_delta_circ
BULGE_MAX_SPECIFIC_ENERGY = -0.6
SUFFIX = "_02"

In [5]:
def read_data(simulation: str) -> pd.DataFrame:
    """
    Return a DataFrame with data of interest.
    """
    settings = Settings()

    s = Snapshot(simulation=simulation, loadonlytype=[0, 1, 2, 3, 4, 5])
    s.tag_particles_by_region(
        disc_std_circ=DISC_STD_CIRC,
        disc_min_circ=DISC_MIN_CIRC,
        cold_disc_delta_circ=COLD_DISC_DELTA_CIRC,
        bulge_max_specific_energy=BULGE_MAX_SPECIFIC_ENERGY)

    is_real_star = (s.type == 4) & (s.stellar_formation_time > 0)
    is_main_obj = (s.halo == s.halo_idx) & (s.subhalo == s.subhalo_idx)

    df = pd.DataFrame()
    df["RegionTag"] = s.region_tag[is_real_star & is_main_obj]
    df["Mass_Msun"] = s.mass[is_real_star & is_main_obj]

    return df

In [6]:
def get_statistics(df: pd.DataFrame) -> tuple:
    total_solar_masses = df["Mass_Msun"].sum()
    components_solar_masses = np.zeros(4)
    for i in range(len(settings.components)):
        components_solar_masses[i] = df["Mass_Msun"][
            df["RegionTag"] == i].sum()
    return components_solar_masses / total_solar_masses

In [15]:
def plot_galaxy(simulation: str) -> None:
    settings = Settings()

    fig, ax = plt.subplots(figsize=(2.5, 2.5))

    ax.tick_params(which='both', direction="in")

    ax.set_xlim(-0.5, 3.5)
    ax.set_xticks([0, 1, 2, 3])
    ax.set_xticklabels([])

    ax.set_ylim(0, 1)
    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8, 1])
    ax.set_ylabel(r"$f_\star$")

    label = f"Au{parse(simulation)[0]}"
    data = get_statistics(read_data(simulation))

    for j in range(len(data)):
        ax.bar(x=j, height=data[j],
               color=list(settings.component_colors.values())[j],
               width=0.5, linewidth=0)
        ax.text(j, data[j] + 0.025,
                s=r"$\textbf{" + str(int(np.round(100 * data[j], 0))) \
                    + "\%" + "}$",
                c=list(settings.component_colors.values())[j],
                ha="center", va="bottom", size=8.0)
        ax.text(j, -0.05, size=8.0,
                s=r"$\textbf{" + settings.components[j] + "}$",
                c=list(settings.component_colors.values())[j],
                ha="center", va="top")

    ax.text(
        x=0.05, y=0.95, size=8.0,
        s=r"$\texttt{" + label + "}$",
        ha="left", va="top", transform=ax.transAxes)

    fig.savefig(
        f"../images/galaxy_decomposition/"
        f"stellar_mass_distribution_{simulation}{SUFFIX}.pdf")
    plt.close(fig)

In [16]:
plot_galaxy("au6_or_l4_s127")

In [15]:
simulations = [f"au{i}_or_l4_s127" for i in settings.galaxies]

In [16]:
fig, axs = plt.subplots(figsize=(7.4, 7.2), nrows=6, ncols=5,
                        sharey=True, sharex=True,
                        gridspec_kw={"hspace": 0.0, "wspace": 0.0})

for ax in axs.flat:
    ax.tick_params(which='both', direction="in")
    ax.set_xlim(-0.5, 3.5)
    ax.set_xticks([0, 1, 2, 3])
    ax.set_xticklabels([])
    ax.set_ylim(0, 1)
    ax.set_yticks([0, 0.2, 0.4, 0.6, 0.8])
    ax.set_ylabel(r"$f_\star$")
    ax.label_outer()

for i, simulation in enumerate(simulations):
    data = get_statistics(read_data(simulation))
    current_ax = axs.flatten()[i]
    current_ax.text(
        x=0.05, y=0.95, size=6.0,
        s=r"$\texttt{" + simulation.upper() + "}$",
        ha="left", va="top", transform=current_ax.transAxes)

    for j in range(len(data)):
        current_ax.bar(x=j, height=data[j],
                       color=list(settings.component_colors.values())[j],
                       width=0.5, linewidth=0)
        current_ax.text(j, data[j] + 0.025,
            s=r"$\textbf{" + str(int(np.round(100 * data[j], 0))) \
                + "\%" + "}$",
            c=list(settings.component_colors.values())[j],
            ha="center", va="bottom", size=5.0)
        
        if current_ax.get_subplotspec().is_last_row():
            current_ax.text(j, -0.05, size=6.0,
                            s=r"$\textbf{" + settings.components[j] + "}$",
                            c=list(settings.component_colors.values())[j],
                            ha="center", va="top")

    fig.savefig(
        f"../images/galaxy_decomposition/"
        f"stellar_mass_distribution{SUFFIX}.pdf")
    plt.close(fig)